In [6]:
import sys, os
sys.path.append('/home/mengqing/InterDiff/interdiff')
from psbody.mesh.colors import name_to_rgb
from omegaconf import OmegaConf
from psbody.mesh import Mesh   #主要是这个库进行静态可视化，一个待观察序列有T帧，选取其中一帧的mesh看
import meshplot as mp
import numpy as np
import smplx
import torch
import mano
import yaml
import trimesh
from os.path import isfile
from libsmpl.smplpytorch.pytorch.smpl_layer import SMPL_Layer  #这个SMPLmodel作用是通过smpl参数得到mesh，进而可视化。
from scipy.spatial.transform import Rotation
from tqdm import tqdm
import json
with open( "/home/mengqing/InterDiff/interdiff/data/cfg/BEHAVE.yml", 'r') as stream:
    paths = yaml.safe_load(stream)
SPLIT_PATH, OBJECT_PATH, MOTION_PATH, MODEL_PATH = paths['SPLIT_PATH'], paths['OBJECT_TEMPLATE'], paths['MOTION_PATH'], paths['MODEL_PATH']

In [10]:
import igl
class ContactLabelGenerator(object):
    "class to generate contact labels"
    def __init__(self):
        pass
        

    def get_contact_labels(self, smpl, object_points, thres=0.02):
        """
        sample point on the object surface and compute contact labels for each point
        :param smpl: trimesh object
        :param obj: trimesh object
        :param num_samples: number of samples on object surface
        :param thres: threshold to determine whether a point is in contact with the human
        :return:
        for each point: a binary label (contact or not) and the closest SMPL vertex
        """
        dist, faces, vertices, normals = igl.signed_distance(object_points, smpl.vertices, smpl.faces, return_normals=True)

        # smpl_all = np.concatenate([vertices, normals], axis=1)
        contact_object_label = np.where(dist<thres)[0]
        # print(dist<thres, object_points[np.where(dist<thres)[0]].shape)
        # smpl.vertices - object_points[np.where(dist<thres)[0]]
        vertices_1 = np.expand_dims(object_points[contact_object_label], axis=0)
        vertices_2 = np.expand_dims(smpl.vertices, axis=1)

        contact_human_label = np.where((np.linalg.norm((vertices_1 - vertices_2), axis=2) < thres).any(axis=1))[0]
        return contact_object_label, contact_human_label

    def to_trimesh(self, mesh):
        tri = trimesh.Trimesh(mesh.v, mesh.f, process=False)
        return tri
markerset_ssm67_smplh = [3470, 3171, 3327, 857, 1812, 628, 182, 3116, 3040, 239,
                         1666, 1725, 0, 2174, 1568, 1368, 3387, 2112, 1053, 1058,
                         3336, 3346, 1323, 2108, 3122, 3314, 1252, 1082, 1861, 1454,
                         850, 2224, 3233, 1769, 6728, 4343, 5273, 4116, 3694, 6399,
                         6540, 6488, 3749, 5135, 5194, 3512, 5635, 5210, 4360, 4841,
                         6786, 5573, 4538, 4544, 6736, 6747, 4804, 5568, 6544, 6682,
                         5322, 4927, 5686, 4598, 6633, 3506, 3508]

In [11]:
name = 'Date07_Sub08_toolbox'
outfile = os.path.join(MOTION_PATH, name, 'contact.npz')
#下载object 和 human 
with np.load(os.path.join(MOTION_PATH, name, 'object_fit_all.npz'), allow_pickle=True) as f:
    obj_angles, obj_trans, frame_times = f['angles'], f['trans'], f['frame_times']
with np.load(os.path.join(MOTION_PATH, name, 'smpl_fit_all.npz'), allow_pickle=True) as f:
    poses, betas, trans = f['poses'], f['betas'], f['trans']

info_file = os.path.join(MOTION_PATH, name, 'info.json')
info = json.load(open(info_file))
gender = info['gender']
obj_name = info['cat']
batch_end = len(frame_times)
print(frame_times[0])
#object 模板，不是以原点为中心
mesh_obj = Mesh()
mesh_obj.load_from_obj(os.path.join(OBJECT_PATH, f"{obj_name}/{obj_name}.obj"))
mp_viewer = mp.plot(mesh_obj.v, mesh_obj.f, name_to_rgb['pink'])
beginning = np.array([[0, 0,0]])
ending =  np.array([[1, 0, 0]])
ending2 = np.array([[0, 1,0]])
ending3 = np.array([[0, 0,1]])
mp_viewer.add_lines(beginning, ending,)
#mp_viewer.add_points(ending,name_to_rgb['red'])
mp_viewer.add_lines(beginning, ending2)
mp_viewer.add_lines(beginning, ending3)
shading = {}
shading["point_size"] = 0.5
mp_viewer.add_points(ending,c="red",shading=shading)
mp_viewer.add_points(ending2,c="green",shading=shading)
mp_viewer.add_points(ending3,c="blue",shading=shading)

smpl_male = SMPL_Layer(center_idx=0, gender='male', num_betas=10,
                            model_root=str(MODEL_PATH), hands=True)
smpl_female = SMPL_Layer(center_idx=0, gender='female', num_betas=10,
                        model_root=str(MODEL_PATH), hands=True)
smpl = {'male': smpl_male, 'female': smpl_female}[gender]
verts, jtr, _, _ = smpl(torch.tensor(poses), th_betas=torch.tensor(betas), th_trans=torch.tensor(trans))
verts = np.float32(verts)
faces = smpl.th_faces.numpy()
pelvis = np.float32(jtr[:, 0])

print('verts',verts.shape)
print('pelvis',pelvis.shape)

obj_verts = mesh_obj.v
obj_faces = mesh_obj.f
center = np.mean(obj_verts, 0)
obj_verts = obj_verts - center
obj = trimesh.Trimesh(obj_verts, obj_faces, process=False)
#mp_viewer.add_mesh(obj_verts,mesh_obj.f, name_to_rgb["green"])

object_points, object_faces = obj.sample(2048, return_index=True)
#mp_viewer.add_points(object_points,c = 'yellow')

# for idx in tqdm(range(batch_end)): 
idx=100
if idx == 100:
    #objrct at start 
    obj_v = object_points.copy()
    a, t = obj_angles[idx], obj_trans[idx]
    rot = Rotation.from_rotvec(a).as_matrix()
    obj_v = np.matmul(obj_v, rot.T) + t
    #mp_viewer.add_points(obj_v,c = 'red')
    #human at start 
    mp_viewer.add_mesh(verts[idx], faces, name_to_rgb["green"])
    joints = jtr[idx]

    smplsmpl = trimesh.Trimesh(verts[idx], faces, process=False)
    generator = ContactLabelGenerator()
    contact_object_label, contact_human_label = generator.get_contact_labels(
    smplsmpl, obj_v)
    mp_viewer.add_points(obj_v[contact_object_label],c = 'red')

    #calculate rotation
    centroid = pelvis[idx]
    print('centroid',centroid)
    print('torch.tensor(poses)[idx,:3]',torch.tensor(poses)[idx,:3])
    global_orient = Rotation.from_rotvec(torch.tensor(poses)[idx,:3]).as_matrix()
    rotation_v = np.eye(3).astype(np.float32)
    cos, sin = global_orient[0, 0] / np.sqrt(global_orient[0, 0]**2 + global_orient[2, 0]**2), global_orient[2, 0] / np.sqrt(global_orient[0, 0]**2 + global_orient[2, 0]**2)
    rotation_v[[0, 2, 0, 2], [0, 2, 2, 0]] = np.array([cos, cos, -sin, sin])
    rotation = np.linalg.inv(rotation_v).astype(np.float32)

#idx =idx+1
print('original_trans',trans[idx])   #人体的trans 不等于盆骨点jtr[:, 0]
new_trans= trans[idx] - centroid
print('new_trans',new_trans)
new_pelvis = pelvis[idx] - centroid
print("new_pelvis",new_pelvis)
pelvis_original = new_pelvis - new_trans  # pelvis position in original smpl coords system
print("pelvis_original",pelvis_original)
new_trans = np.dot(new_trans + pelvis_original, rotation.T) - pelvis_original
pelvis1 = np.dot(new_pelvis, rotation.T)

# human vertex in the canonical system
human_verts_tran = verts[idx] - centroid
human_verts_tran = np.dot(human_verts_tran, rotation.T)
shading ={}
shading["point_size"] = 0.2
mp_viewer.add_points(human_verts_tran[markerset_ssm67_smplh],c = 'black',shading = shading)

poseidx=poses[idx]
print('original_poseidx[:3]',poseidx[:3])
r_ori = Rotation.from_rotvec(torch.tensor(poseidx)[:3])
r_new = Rotation.from_matrix(rotation) * r_ori
poseidx[:3] = r_new.as_rotvec()
print('poseidx[:3]',poseidx[:3])
print(torch.tensor(poseidx).shape)
print(torch.tensor(betas)[idx].shape)
print(torch.tensor(new_trans).shape)
verts, jtr, _, _ = smpl(torch.tensor(poseidx).unsqueeze(0), th_betas=torch.tensor(betas)[idx].unsqueeze(0), th_trans=torch.tensor(new_trans).unsqueeze(0))
# verts00, jtr00, _, _ = smpl(torch.tensor(poseidx).unsqueeze(0)-torch.tensor(poseidx).unsqueeze(0), th_betas=torch.tensor(betas)[idx].unsqueeze(0), th_trans=torch.tensor(new_trans).unsqueeze(0)-torch.tensor(new_trans).unsqueeze(0))
# verts00 = np.float32(verts00)
verts = np.float32(verts)
mp_viewer.add_mesh(verts[0], faces, name_to_rgb["green"])
#mp_viewer.add_mesh(verts00[0], faces, name_to_rgb["red"])

#objects
new_obj_trans = obj_trans[idx] - centroid
new_obj_trans = np.dot(new_obj_trans, rotation.T)

r_ori = Rotation.from_rotvec(obj_angles[idx])
r_new = Rotation.from_matrix(rotation) * r_ori
new_obj_angle = r_new.as_rotvec()
print('new_obj_angle',new_obj_angle)
obj_points = object_points.copy()  #这里的物体点云是normalize到原点的

rot = r_new.as_matrix()
obj_points = np.matmul(obj_points, rot.T) + new_obj_trans
mp_viewer.add_points(obj_points,c = 'red')


    



t0003.000


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.6777454…

verts (926, 6890, 3)
pelvis (926, 3)
centroid [0.1891817  0.38335225 2.1849098 ]
torch.tensor(poses)[idx,:3] tensor([-3.0400,  0.0564, -1.0521])
original_trans [0.19059327 0.579813   2.165118  ]
new_trans [ 0.00141157  0.19646075 -0.01979184]
new_pelvis [0. 0. 0.]
pelvis_original [-0.00141157 -0.19646075  0.01979184]
original_poseidx[:3] [-3.0399659   0.05644289 -1.0520763 ]
poseidx[:3] [ 3.0584848e+00 -1.2769640e-02 -5.3080736e-04]
torch.Size([156])
torch.Size([10])
torch.Size([3])
new_obj_angle [-0.86430345  2.47985796  1.20933548]


11